![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_AlbertForQuestionAnswering.ipynb)

## Import ONNX AlbertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `AlbertForQuestionAnswering` is only available since in `Spark NLP 5.1.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import ALBERT models trained/fine-tuned for question answering via `AlbertForQuestionAnswering`. These models are usually under `Question Answering` category and have `albert` in their labels
- Reference: [TFAlbertForQuestionAnswering](https://huggingface.co/transformers/model_doc/albert#transformers.TFAlbertForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=albert&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [twmkn9/albert-base-v2-squad2](https://huggingface.co/twmkn9/albert-base-v2-squad2) model from HuggingFace as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.
- In addition to the ALBERT model, we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [4]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering

MODEL_NAME = "twmkn9/albert-base-v2-squad2"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

Some weights of the model checkpoint at twmkn9/albert-base-v2-squad2 were not used when initializing AlbertForQuestionAnswering: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


('onnx_models/twmkn9/albert-base-v2-squad2/tokenizer_config.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/special_tokens_map.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/spiece.model',
 'onnx_models/twmkn9/albert-base-v2-squad2/added_tokens.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -l {EXPORT_PATH}

total 46748
-rw-r--r-- 1 root root      844 Jun  7 05:40 config.json
-rw-r--r-- 1 root root 44817937 Jun  7 05:40 model.onnx
-rw-r--r-- 1 root root      286 Jun  7 05:40 special_tokens_map.json
-rw-r--r-- 1 root root   760289 Jun  7 05:40 spiece.model
-rw-r--r-- 1 root root     2616 Jun  7 05:40 tokenizer_config.json
-rw-r--r-- 1 root root  2275547 Jun  7 05:40 tokenizer.json


- We need to move the `spiece.model` file from the tokenizer into an assets folder, as this is where Spark NLP looks for it when working with models like Albert or other SentencePiece-based tokenizers.

In [6]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/spiece.model {EXPORT_PATH}/assets

In [7]:
!ls -lR {EXPORT_PATH}

onnx_models/twmkn9/albert-base-v2-squad2:
total 46008
drwxr-xr-x 2 root root     4096 Jun  7 05:41 assets
-rw-r--r-- 1 root root      844 Jun  7 05:40 config.json
-rw-r--r-- 1 root root 44817937 Jun  7 05:40 model.onnx
-rw-r--r-- 1 root root      286 Jun  7 05:40 special_tokens_map.json
-rw-r--r-- 1 root root     2616 Jun  7 05:40 tokenizer_config.json
-rw-r--r-- 1 root root  2275547 Jun  7 05:40 tokenizer.json

onnx_models/twmkn9/albert-base-v2-squad2/assets:
total 744
-rw-r--r-- 1 root root 760289 Jun  7 05:40 spiece.model


Voila! We have our `spiece.model` inside assets directory

## Import and Save AlbertForQuestionAnswering in Spark NLP


Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 45.8 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `AlbertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [10]:
from sparknlp.annotator import AlbertForQuestionAnswering

spanClassifier = AlbertForQuestionAnswering.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your AlbertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 44524
-rw-r--r-- 1 root root 44824911 Jun  7 05:44 albert_classification_onnx
-rw-r--r-- 1 root root   760289 Jun  7 05:44 albert_spp
drwxr-xr-x 2 root root     4096 Jun  7 05:44 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [14]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import AlbertForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

qa_model = AlbertForQuestionAnswering.load(f"{MODEL_NAME}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")

pipeline = Pipeline(stages=[
    document_assembler,
    qa_model
])

example = spark.createDataFrame([
    ["What's my name?", "My name is Clara and I live in Berkeley."]
], ["question", "context"])

result = pipeline.fit(example).transform(example)
result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[clara]|
+-------+



That's it! You can now go wild and use hundreds of `AlbertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
